# Optimizing Modeling

* Everthing is in g/l
* PET a is 0.88
* Total Enzyme(E): 0.0000844 g/L
* Susbstrate(S) Data"
    * PET = 2.04 g/L, PBT fil = 10.63, PBT fib = 2.33, PTT fil = 4.66, PTT fib = 2.14 g/L

* Since PBT and PTT each have 2 different concentrations for the 2 sets of data, use each initial concentration into the solver and find the average of each sets of data, then rmsd those results
* New optimization/costfunction process:
    * Have the cost function put the fiber variables through the fiber data and rsmd that, then do the same with filament, then find the average rmsd 
    * Optimize the fiber variables first with the fiber data (don't use averaged data), then use the k values to put it through the differential equations again with the filament variables as well to obtain the overall solution
* Save the Kin from PET as the constant kin for everything else
* Split the fiber and filament data as separate
* Use kinematics from PET as constant.

In [2]:
import Pkg
using DiffEqBase, OrdinaryDiffEq, DifferentialEquations, Catalyst, GraphViz, Plots, Optim, StatsBase, CSV, DataFrames, Pkg; Pkg.add("DifferentialEquations")
plotly()

    Updating registry at `C:\Users\gordo\.julia\registries\General`
    Updating git-repo `https://github.com/JuliaRegistries/General.git`
   Resolving package versions...
  No Changes to `C:\Users\gordo\.julia\environments\v1.6\Project.toml`
  No Changes to `C:\Users\gordo\.julia\environments\v1.6\Manifest.toml`


Plots.PlotlyBackend()

In [3]:
#Loading the TPA production data into ModelData
using DataFrames
ModelData = CSV.read("Modeling_Data.csv", DataFrame, normalizenames = true)

,Column1,PET_Purified_Filament,PET_Purified_20_Mesh,PBT_Purified_Filament
,String31,Float64,Float64,Float64
1,0,0.0,0.0,0.0
2,24,0.148323,0.751492,0.00307528
3,48,2.9738,1.04542,0.0026613
4,120,7.58187,1.06582,0.0044355
5,168,7.69453,1.08918,0.00337098
6,Enzyme Amount (mL),0.06,0.03,0.06
7,Enzyme uM,0.1,0.1,0.1
8,final enzyme conc. (uM),0.00291262,0.00147783,0.00291262


In [4]:
#Assigning each polymer sample data with to a variable
PET_Pure_Fil = identity.(ModelData[1:5,"PET_Purified_Filament"])
PET_Pure_Mesh = identity.(ModelData[1:5, "PET_Purified_20_Mesh"])
PBT_Pure_Fil = identity.(ModelData[1:5,"PBT_Purified_Filament"])
PBT_Pure_Mesh = identity.(ModelData[1:5,"PBT_Purified_20_Mesh"])
PTT_Pure_Fil = identity.(ModelData[1:5,"PTT_Purified_Filament"])
PTT_Pure_Mesh = identity.(ModelData[1:5,"PTT_Purified_20_Mesh"])
PPET_Pure_Fil = identity.(ModelData[1:5,"PPET_Purified_Filament"])

5-element Vector{Float64}:
 0.0
 0.48471144
 0.93849266
 1.05618126
 1.08397706

### PET Differential Equation

In [5]:
#Differnetial equation for PET; follows the differential equations in the readme
function PETDiffEq!(du,u,p,t)
 du[1] = -((p[1]*0.0000844*u[1])/(p[2]+u[1]))*ℯ^(-p[3]*t)
 du[2] = -0.86*du[1]
end

PETDiffEq! (generic function with 1 method)

### Cost Function

In [6]:
#=Cost Function which generates the model with the differnetial equation and then
calculates the root-mean-square deviation of the actual data with the model=# 
function costFunction(polymerData, u0_1, DiffEq!, x) # x is the S0 with reaction kinectics [s0, k1, k-1, k2]
    global kValues
    
    prob1 = ODEProblem(DiffEq!,u0_1,tspan,x)
    sol1 = solve(prob1, Tsit5())
    sol1 = [sol1(i)[2] for i in reactionTime]
    rmsd1 = rmsd(polymerData[2], sol1)
    
    kValues = x
    println(x)
    println(rmsd1)

    return rmsd1

end

costFunction (generic function with 1 method)

### PET Data

In [7]:
#Instantiates the necessary variables for PET
reactionTime = [0, 22, 46, 120, 168]
avePETexp_data = [(PPET_Pure_Fil[i] + PET_Pure_Mesh[i])/2 for i in 1:length(PPET_Pure_Fil)]
avePETexp_data = (reactionTime, avePETexp_data)
PPETexp_data = (reactionTime, PPET_Pure_Fil)
PETFIBexp_data = (reactionTime, PET_Pure_Mesh)


PETx = [650, 0.0001, 0.05] #[Kc(k2), Km, k(kin)]
tspan = (0.0,168.0)
PETu0 = [2.04,0.0]

2-element Vector{Float64}:
 2.04
 0.0

### PET reaction kinetics optimized and modeled

In [8]:
#Optimizes the k-values of the PET data using gradient descent
PETtest = 0
PETtest2 = 0
PETtestx = PETx
lower = [0.0, 0.0, 0.0]
upper = [Inf, Inf, Inf]
inner_optimizer = GradientDescent()

while true
    PETtest = Optim.optimize(PETtestx -> costFunction(PETFIBexp_data, PETu0, PETDiffEq!, PETtestx), lower, upper, PETtestx,  Fminbox(inner_optimizer), Optim.Options(iterations = 3000))

    if abs(sum(Optim.minimizer(PETtest)) - sum(PETtestx)) <= 1
        break
    end
    println("rep")

    PETtestx = Optim.minimizer(PETtest)

end
#PET_k = Optim.minimizer(PETtest)

[650.0039360453941, 0.0001, 0.05]
0.1343899641812139
[649.9960639546059, 0.0001, 0.05]
0.13439894873538466
[650.0, 0.00010605545445239335, 0.05]
0.134397498429429
[650.0, 9.394454554760666e-5, 0.05]
0.13439141447437805
[650.0, 0.0001, 0.05000605545445239]
0.1344686334176251
[650.0, 0.0001, 0.04999394454554761]
0.13432027251179587
[650.0, 0.0001, 0.05]
0.1343944564376854
[650.0039360453941, 0.0001, 0.05]
0.1343899641812139
[649.9960639546059, 0.0001, 0.05]
0.13439894873538466
[650.0, 0.00010605545445239335, 0.05]
0.134397498429429
[650.0, 9.394454554760666e-5, 0.05]
0.13439141447437805
[650.0, 0.0001, 0.05000605545445239]
0.1344686334176251
[650.0, 0.0001, 0.04999394454554761]
0.13432027251179587
[650.0, 0.0001, 0.05]
0.1343944564376854
[650.0039371867203, 9.618322003143842e-5, 0.037756071616412784]
0.10619682310348817
[649.9960650959183, 9.618322003143842e-5, 0.037756071616412784]
0.10618864841122154
[650.0000011413193, 0.00010223867448383176, 0.037756071616412784]
0.10618914867918186


Excessive output truncated after 524310 bytes.

[650.0001074958842, 2.4270770419457705e-5, 0.042237275256127214]
0.056935470185605026
[650.0001074958842

LoadError: InterruptException:

In [9]:
PET_k = kValues

3-element Vector{Float64}:
 650.0004927925376
   8.428115633074904e-6
   0.04223777451503561

In [10]:
#Plots the PET results
PETplot = ODEProblem(PETDiffEq!,PETu0,tspan,PET_k)
PET_rxRes = solve(PETplot, Tsit5())

PET_rxRes_plot = plot(PET_rxRes, lw=2, title = "PET Model")
display(plot!(PETFIBexp_data,seriestype = :scatter,label = "PET data"))

print("PET results:")
println("rmsd: " , costFunction(PETFIBexp_data, PETu0, PETDiffEq!, PET_k))

PET results:[650.0004927925376, 8.428115633074904e-6, 0.04223777451503561]
0.0569336709599699
rmsd: 0.0569336709599699


### PET product model only

In [11]:
#Isolates the TPA values for the plot
PETlist = []
for i in 0:168
    PETlist = vcat(PETlist, PET_rxRes(i)[2])
end
petplot1 = plot([0:1:168;],PETlist, lw=2, title = "PET Model",color = "black")
petplot1.series_list[1][:label] = "Modeled P(t)"
plot!(PETFIBexp_data,seriestype = :scatter,label = "PET data",color = "black",marker=([:diamond :d]))
plot!(xtickfontsize=10,ytickfontsize=10)
ylabel!("TPA (mg/ml)")
xlabel!("Time (Hours)")
plot!(titlefontsize=15, xtickfontsize=12,ytickfontsize=12,xguidefontsize=15,yguidefontsize=15,legend=true)
display(petplot1)

### PBT Diff Equations and Setup

In [12]:
#Differential equations for PBT polymer.
function PBTDiffEq!(du,u,p,t)
 du[1] = -((p[1]*0.0000844*u[1])/(p[2]+u[1]))*ℯ^(-PET_k[3]*t)
 du[2] = -0.65*du[1]
end

PBTDiffEq! (generic function with 1 method)

In [13]:
#Instantiates necessary data for PBT and PTT
PBTFILexp_data = (reactionTime, PBT_Pure_Fil)
PBTFIBexp_data = (reactionTime, PBT_Pure_Mesh)
avePBTexp_data = [(PBT_Pure_Fil[i] + PBT_Pure_Mesh[i])/2 for i in 1:length(PBT_Pure_Fil)]
avePBTexp_data = (reactionTime,avePBTexp_data)
PTTFILexp_data = (reactionTime, PTT_Pure_Fil)
PTTFIBexp_data = (reactionTime, PTT_Pure_Mesh)
avePTTexp_data = [(PTT_Pure_Fil[i] + PTT_Pure_Mesh[i])/2 for i in 1:length(PTT_Pure_Fil)]
avePTTexp_data = (reactionTime,avePTTexp_data)

([0, 22, 46, 120, 168], [0.0, 0.0390324, 0.05044642, 0.05905129, 0.06138732])

In [14]:
PBTtestx = [12.0,15.0,PET_k[3]]
PBTu01 = [10.63,0.0]
PBTu02 = [2.33,0.0]
lower = [0.0, 0.0, 0.0]
upper = [Inf, Inf, Inf]
inner_optimizer = GradientDescent()
#=
PBTtest = Optim.optimize(PBTtestx -> costFunction(PBTFILexp_data, PBTFIBexp_data, PBTu01, PBTu02, PBTDiffEq!, PBTtestx), lower, upper, PBTtestx, Fminbox(inner_optimizer))
=#

GradientDescent{LineSearches.InitialPrevious{Float64}, LineSearches.HagerZhang{Float64, Base.RefValue{Bool}}, Nothing, Optim.var"#11#13"}(LineSearches.InitialPrevious{Float64}
  alpha: Float64 1.0
  alphamin: Float64 0.0
  alphamax: Float64 Inf
, LineSearches.HagerZhang{Float64, Base.RefValue{Bool}}
  delta: Float64 0.1
  sigma: Float64 0.9
  alphamax: Float64 Inf
  rho: Float64 5.0
  epsilon: Float64 1.0e-6
  gamma: Float64 0.66
  linesearchmax: Int64 50
  psi3: Float64 0.1
  display: Int64 0
  mayterminate: Base.RefValue{Bool}
, nothing, Optim.var"#11#13"(), Flat())

### PBT reaction kinetics optimized and modeled

In [15]:
#Optimizes the K-values for PBT
PBTtest1 = 0
PBTu01 = [10.63,0.0]
PBTu02 = [2.33,0.0]
lower = [0.0, 0.0, 0.0]
upper = [Inf, Inf, Inf]

while true
    PBTtest1 = Optim.optimize(PBTtestx -> costFunction(PBTFIBexp_data, PBTu02, PBTDiffEq!, PBTtestx), lower, upper, PBTtestx, Fminbox(inner_optimizer))
    
    if abs(sum(Optim.minimizer(PBTtest1)) - sum(PBTtestx)) <= 1
        break
    end

    println("rep")
    PBTtestx = Optim.minimizer(PBTtest1)
end

PBT_k = vcat(kValues[1:2], PET_k[3])

[12.000072665453429, 15.0, 0.04223777451503561]
0.0029237776570312703
[11.999927334546571, 15.0, 0.04223777451503561]
0.002923797469197638
[12.0, 15.000090831816786, 0.04223777451503561]
0.002923796138028533
[12.0, 14.999909168183214, 0.04223777451503561]
0.0029237789881168333
[12.0, 15.0, 0.042243829969488]
0.0029237875631168185
[12.0, 15.0, 0.04223171906058322]
0.0029237875631168185
[12.0, 15.0, 0.04223777451503561]
0.0029237875631168185
[12.000072665453429, 15.0, 0.04223777451503561]
0.0029237776570312703
[11.999927334546571, 15.0, 0.04223777451503561]
0.002923797469197638
[12.0, 15.000090831816786, 0.04223777451503561]
0.002923796138028533
[12.0, 14.999909168183214, 0.04223777451503561]
0.0029237789881168333
[12.0, 15.0, 0.042243829969488]
0.0029237875631168185
[12.0, 15.0, 0.04223171906058322]
0.0029237875631168185
[12.0, 15.0, 0.04223777451503561]
0.0029237875631168185
[12.000208991607657, 14.999905595885771, 0.04223800379045222]
0.002923750160030616
[12.000063659049776, 14.99990

Excessive output truncated after 524303 bytes.

[21.72352259778622, 8.995786424764114, 0.21019493969669847]
0.000298448541769272
[21.723654144384433, 8.995840898339074, 0.21019493969669847]


3-element Vector{Float64}:
 21.69913918705604
  8.982978723512229
  0.04223777451503561

In [16]:
PBT_k = vcat(kValues[1:2], PET_k[3])

3-element Vector{Float64}:
 21.69913918705604
  8.982978723512229
  0.04223777451503561

In [17]:
#Plots PBT results
PBTprob = ODEProblem(PBTDiffEq!,PBTu02,tspan,PBT_k)
PBTsol = solve(PBTprob, Tsit5())

PBT_rxRes_plot = plot(PBTsol, lw=2, title = "PBT Model")
display(plot!(PBTFIBexp_data,seriestype = :scatter,label = "PBT data"))

print("PBT results:")
println(costFunction(PBTFIBexp_data, PBTu02, PBTDiffEq!, PBT_k))

PBT results:[21.69913918705604, 8.982978723512229, 0.04223777451503561]
0.00029844838345000427
0.00029844838345000427


### PBT product model only

In [18]:
#Isolates TPA for PBT of the plot
plotPBT_rxRes1 = [PBTsol[i][2] for i in 1:length(PBTsol)]

PBTlist = []
for i in 0:168
    PBTlist = vcat(PBTlist, PBTsol(i)[2])
end
[0:1:168;]


pl1 = plot([0:1:168;], PBTlist, lw=2, title = "PBT Model",color = "black")
pl1.series_list[1][:label] = "Modeled P(t)"
plot!(PBTFIBexp_data,seriestype = :scatter,label = "PBT data",color = "black", marker=([:utriangle :d]))
plot!(xtickfontsize=10,ytickfontsize=10)
ylabel!("TPA (mg/ml)")
xlabel!("Time (Hours)")
plot!(titlefontsize=15, xtickfontsize=12,ytickfontsize=12,xguidefontsize=15,yguidefontsize=15,legend=true)
display(pl1)

### PTT reaction kinetics optimized and modeled|

In [19]:
#Differential equations for PTT
function PTTDiffEq!(du,u,p,t)
 du[1] = -((p[1]*0.0000844*u[1])/(p[2]+u[1]))*ℯ^(-PET_k[3]*t)
 du[2] = -0.81*du[1]
end

PTTDiffEq! (generic function with 1 method)

In [20]:
#Optimizes K-values for PTT
PTTtest1 = 0
PTTtest2 = 0
PTTtestx = [50.0, 1.0, PET_k[3]]
PTTu01 = [4.66,0.0]
PTTu02 = [2.14,0.0]
lower = [0.0, 0.0, 0.0]
upper = [Inf, Inf, Inf]

while true
    PTTtest1 = Optim.optimize(PTTtestx -> costFunction(PTTFIBexp_data, PTTu02, PTTDiffEq!, PTTtestx), lower, upper, PTTtestx, Fminbox(inner_optimizer))
    
    if abs(sum(Optim.minimizer(PTTtest1)) - sum(PTTtestx)) <= 1
        break
    end

    println("rep")
    PTTtestx = Optim.minimizer(PTTtest1)
end

PTT_k = vcat(kValues[1:2],PET_k[3])

[50.00030277272262, 1.0, 0.04223777451503561]
0.02262383415906323
[49.99969722727738, 1.0, 0.04223777451503561]
0.02262435091362492
[50.0, 1.0000060554544523, 0.04223777451503561]
0.02262417563682372
[50.0, 0.9999939445455476, 0.04223777451503561]
0.022624009435640507
[50.0, 1.0, 0.042243829969488]
0.022624092536352374
[50.0, 1.0, 0.04223171906058322]
0.022624092536352374
[50.0, 1.0, 0.04223777451503561]
0.022624092536352374
[50.00030277272262, 1.0, 0.04223777451503561]
0.02262383415906323
[49.99969722727738, 1.0, 0.04223777451503561]
0.02262435091362492
[50.0, 1.0000060554544523, 0.04223777451503561]
0.02262417563682372
[50.0, 0.9999939445455476, 0.04223777451503561]
0.022624009435640507
[50.0, 1.0, 0.042243829969488]
0.022624092536352374
[50.0, 1.0, 0.04223171906058322]
0.022624092536352374
[50.0, 1.0, 0.04223777451503561]
0.022624092536352374
[50.00115616009987, 0.9862773357558288, 0.04225174446605068]
0.022433952504881186
[50.0005506043194, 0.9862773357558288, 0.04225174446605068]


Excessive output truncated after 524289 bytes.

]
0.00250374293962558
[52.03674210124556, 0.017903255506301532, 0.09877157819034003]
0.00250374293962558
[52.03674210124556, 0.017903255506301532, 0.09877763364479243]
0.00250374293962558
[52.03712884283998, 0.018220267647990708, 0.09887967130782385]
0.002503763396310889
[52.036498629729124, 0.018220267647990708, 0.09887967130782385]
0.0025037691297562633
[52.03681373628455, 0.018226323102443102, 0.09887967130782385]
0.002503767059387335
[52.03681373628455, 0.018214212193538314, 0.09887967130782385]
0.002503765417528856
[52.03681373628455, 0.018220267647990708, 0.09888572676227625]
0.0025037662316649088
[52.03681373628455, 

LoadError: InterruptException:

In [21]:
#=
while true
    PTTtest1 = Optim.optimize(PTTtestx -> costFunction(PTTFILexp_data, PTTu02, PTTDiffEq!, PTTtestx), lower, upper, PTTtestx, Fminbox(inner_optimizer))
    
    if abs(sum(Optim.minimizer(PTTtest1)) - sum(PTTtestx)) <= 1
        break
    end

    println("rep")
    PTTtestx = Optim.minimizer(PTTtest1)
end
=#

In [22]:
PTT_k = vcat(kValues[1:2],PET_k[3])

3-element Vector{Float64}:
 52.08043924369814
  0.019628128699921553
  0.04223777451503561

In [23]:
#Plots PTT results
PTTprob = ODEProblem(PTTDiffEq!,PTTu02,tspan,PTT_k)
PTTsol = solve(PTTprob, Tsit5())

PTT_rxRes_plot = plot(PTTsol, lw=2, title = "PTT Model")
display(plot!(PTTFIBexp_data,seriestype = :scatter,label = "PTT data"))

println("PTT results:")
println("Filament rmsd: " , costFunction(PTTFIBexp_data, PTTu02, PTTDiffEq!, PTT_k))

PTT results:
[52.08043924369814, 0.019628128699921553, 0.04223777451503561]
0.002503585444569834
Filament rmsd: 0.002503585444569834


### PTT product model only

In [24]:
#Isolates TPA values for PTT plot
plotPTT_rxRes1 = [PTTsol[i][2] for i in 1:length(PTTsol)]

PTTlist = []
for i in 0:168
    PTTlist = vcat(PTTlist, PTTsol(i)[2])
end

PTTpl1 = plot([0:1:168;],PTTlist, lw=2, title = "PTT Model",color = "black")

PTTpl1.series_list[1][:label] = "Modeled P(t)"
plot!(PTTFIBexp_data,seriestype = :scatter,label = "PTT data",color = "black", marker=([:rect :d]))
plot!(xtickfontsize=10,ytickfontsize=10)
ylabel!("TPA (mg/ml)")
xlabel!("Time (Hours)")
plot!(titlefontsize=15, xtickfontsize=12,ytickfontsize=12,xguidefontsize=15,yguidefontsize=15,legend=true)
display(PTTpl1)